In [1]:
%load_ext autoreload

%autoreload 2

import numpy as np
import time
from matplotlib import pyplot as plt
import os
import camb
from camb import model



from axionCAMB_and_lin_PS import axionCAMB_wrapper 
from axionCAMB_and_lin_PS import load_cosmology  
from axionCAMB_and_lin_PS import lin_power_spectrum 


from halo_model import HMcode_params
from halo_model import PS_nonlin_cold
from halo_model import PS_nonlin_axion

from axion_functions import axion_params

## comparison MDM to LCDM

##################################################
axionHMcode is running
##################################################
Set-up experiment parameters
computing the non-linear total matter power spectrum in the folowing MDM colsmology 

omega_m = 0.14237 
omega_cdm = 0.108 
omega_ax = 0.012 
omega_b = 0.02237 
m_ax = 1e-25eV 
z = 0.0 
h = 0.674 

--------------------------------------------------
axionCAMB is running. Computes transfer function for cosmology with a axion fraction of 0.1
axionCAMB is running. Computes transfer function for a LCDM cosmology
computation time for axionCAMB: 74 s
--------------------------------------------------
Calculate axion quantities; cut-off mass, central density scale of axion density profile and axion halo mass.


/Users/sophievogt/Documents/PhD_local/projects/axion_projects/axionHMcode/axion_functions/axion_cutoff_mass.py:81: UserWarning: Warning. Redshift z = 0.00 is not in the range [1.0, 8.0] where beta2 is defined. Extrapolate
  warnings.warn("Warning. Redshift z = {:.2f} is not in the range [1.0, 8.0] where beta2 is defined. Extrapolate".format(z))


Create dictionary with parameters of HMCode2020
computation time for calcualting axion quantities: 14 s
--------------------------------------------------
Caluclate non-linear power spectrum in mixed DM cosmology with the halo model


/Users/sophievogt/Documents/PhD_local/projects/axion_projects/axionHMcode/halo_model/PS_nonlin_axion.py:144: RuntimeWarning: invalid value encountered in power
  PS_total_cross = axion_dic['frac_cluster']*((one_halo_term_cross**(alpha_param[1]) + two_halo_term_cross**(alpha_param[1]))**(1/alpha_param[1])) + (1-axion_dic['frac_cluster'])*np.sqrt(PS_ax * PS_cold_nonlin)


Caluclate non-linear power spectrum in LCDM cosmology with the halo model
--------------------------------------------------
Save the non-linear power spectra in a file in the folowing order:
k [h/Mpc], non-lin total matter PS with axions [(Mpc/h)^3] and non-lin total matter PS in LCDM [(Mpc/h)^3]
##################################################
axionHMcode is finished, total computation time without axionCAMB computation: 14 s
##################################################


In [2]:
start = time.time()

print('#' * 50)
print('axionHMcode is running')
print('#' * 50)

################################################################################
# Set-up experiment parameters and run axionCAMB
################################################################################
#print('#' * 50)
print('Set-up experiment parameters')
#print('#' * 50)

#IMPORTANT: give the correct path to the intput file which contains all important cosmological parameter
input_file_path = 'input_file.txt'
try:
    f = open(input_file_path)
except IOError:
    print("Input file not accessible, pleas check the file path")
finally:
    f.close()
    
#IMPORTANT:Change here the path to the axionCAMB executable path directory (second path in the function)
# assumes that thee axionCAMB executable is names .camb
axionCAMB_exe_path = '/Users/sophievogt/Documents/axionCAMB'
if os.path.exists(axionCAMB_exe_path+'/./camb') == False:
    print("executabel axionCAMB is not in the given directory, pleas check the path")
      
    
################################################################################    
# save cosmological parameter in a dictionary 
################################################################################
cosmos = load_cosmology.load_cosmology_input(input_file_path) 
cosmos_LCDM = load_cosmology.load_cosmology_input(input_file_path, LCDM = True)
print("computing the non-linear total matter power spectrum in the folowing MDM colsmology \n")
print("omega_m = {0} \nomega_cdm = {1} \nomega_ax = {2} \nomega_b = {3} \nm_ax = {4}eV \nz = {5} \nh = {6} \n".format(cosmos['omega_m_0'], cosmos['omega_d_0'], cosmos['omega_ax_0'], cosmos['omega_b_0'], 
                                                                                                                          cosmos['m_ax'], cosmos['z'], cosmos['h']))



################################################################################
# Run axionCAMB on mixed and LCDM cosmology 
################################################################################
print('-' * 50)
print("axionCAMB is running. Computes transfer function for cosmology with a axion fraction of {}"
      .format(cosmos['Omega_ax_0']/(cosmos['Omega_ax_0']+cosmos['Omega_d_0'])))
axionCAMB_wrapper.axioncamb_params('paramfiles/paramfile_axionCAMB.txt', 
                                   cosmos, output_root='paramfiles/cosmos', print_info = False)
axionCAMB_wrapper.run_axioncamb('paramfiles/paramfile_axionCAMB.txt', 
                                axionCAMB_exe_path, 
                                cosmos, print_info = False)

print("axionCAMB is running. Computes transfer function for a LCDM cosmology")
axionCAMB_wrapper.axioncamb_params('paramfiles/paramfile_axionCAMB_LCDM.txt', 
                                   cosmos_LCDM, output_root='paramfiles/cosmos_LCDM', print_info = False)
axionCAMB_wrapper.run_axioncamb('paramfiles/paramfile_axionCAMB_LCDM.txt', 
                                axionCAMB_exe_path, 
                                cosmos_LCDM, print_info = False)
print("computation time for axionCAMB: {:.0f} s".format(time.time() -start))


start = time.time()
################################################################################
# Create linear power spectra from axionCAMB tranfer functions 
################################################################################
#lin PS on given k range
power_spec_dic_ax = lin_power_spectrum.func_power_spec_dic('paramfiles/cosmos_transfer_out.dat', cosmos)
power_spec_dic_LCDM = lin_power_spectrum.func_power_spec_dic('paramfiles/cosmos_LCDM_transfer_out.dat', cosmos_LCDM)
#interpolated lin PS for the correct computations of the variance

################################################################################
# Compute parameter related to axions and HMCode2020
################################################################################
print('-' * 50)
M_arr = np.logspace(cosmos['M_min'], cosmos['M_max'], 100) 
print("Calculate axion quantities; cut-off mass, central density scale of axion density profile and axion halo mass.")
axion_param = axion_params.func_axion_param_dic(M_arr, cosmos, power_spec_dic_ax, eta_given=False)
print("Create dictionary with parameters of HMCode2020")
hmcode_params = HMcode_params.HMCode_param_dic(cosmos, power_spec_dic_ax['k'], power_spec_dic_ax['power_cold'])


hmcode_params_LCDM = HMcode_params.HMCode_param_dic(cosmos_LCDM, power_spec_dic_LCDM['k'], power_spec_dic_LCDM['power_cold'])
print("computation time for calcualting axion quantities: {:.0f} s".format(time.time() -start))

################################################################################
# Caluclate non-linear power spectrum in mixed DM and LCDM cosmology
################################################################################
print('-' * 50)
print('Caluclate non-linear power spectrum in mixed DM cosmology with the halo model')
PS_matter_nonlin = PS_nonlin_axion.func_full_halo_model_ax(M_arr, power_spec_dic_ax, 
                                                              cosmos, hmcode_params, axion_param,
                                                              alpha = True, 
                                                              eta_given = True, 
                                                              one_halo_damping = True, 
                                                              two_halo_damping = True,
                                                              full_2h = False)

print('Caluclate non-linear power spectrum in LCDM cosmology with the halo model')
PS_LCDM_matter_nonlin = PS_nonlin_cold.func_non_lin_PS_matter(M_arr, power_spec_dic_LCDM['k'], power_spec_dic_LCDM['power_total'], 
                                                              cosmos_LCDM, hmcode_params_LCDM, cosmos_LCDM['Omega_m_0'],
                                                              alpha = True, 
                                                              eta_given = True, 
                                                              one_halo_damping = True, 
                                                              two_halo_damping = True,
                                                              full_2h = False)


################################################################################
# Save both power stepctra in files
################################################################################
print('-' * 50)
print("Save the non-linear power spectra in a file in the folowing order:")
print("k [h/Mpc], non-lin total matter PS with axions [(Mpc/h)^3] and non-lin total matter PS in LCDM [(Mpc/h)^3]")
data_ax = np.column_stack([power_spec_dic_ax['k'], PS_matter_nonlin[0], PS_LCDM_matter_nonlin[0]] )
datafile_path = "nonlin_PS_with_axion.txt" #change path if you want
np.savetxt(datafile_path , data_ax)



print('#' * 50)
print("axionHMcode is finished, total computation time without axionCAMB computation: {:.0f} s".format(time.time() -start))
print('#' * 50)


################################################################################
# Make ratio plot of the two power spectra
################################################################################
plt.semilogx(power_spec_dic_ax['k'], PS_matter_nonlin[0]/PS_LCDM_matter_nonlin[0], label='non-linear ratio', color='r')
plt.semilogx(power_spec_dic_ax['k'], PS_matter_nonlin[1]/PS_LCDM_matter_nonlin[0], label='cold ratio', color='tab:blue')
plt.semilogx(power_spec_dic_ax['k'], PS_matter_nonlin[2]/PS_LCDM_matter_nonlin[0], label='cross ratio', color='tab:orange')
plt.semilogx(power_spec_dic_ax['k'], PS_matter_nonlin[3]/PS_LCDM_matter_nonlin[0], label='axion ratio', color='tab:green')

plt.semilogx(power_spec_dic_ax['k'], power_spec_dic_ax['power_total']/power_spec_dic_LCDM['power_total'], label='linear ratio', linestyle='--', color='r')
plt.axhline(y = 1, linestyle=':', color='black')
plt.margins(x=0)
plt.legend(loc='lower left')
plt.xlabel(r'$k$ [$h/\mathrm{Mpc}$]')
plt.ylabel(r'$P(k)_{\mathrm{mixed\,DM}}/P(k)_{\mathrm{LCDM}}$')
plt.xlim(1e-4, 1e3)
plt.ylim(0.3, 1.1)
plt.show()

## compare LCDM only

In [4]:
# Now get matter power spectra and sigma8 at redshift 0 and 0.8
# parameters can all be passed as a dict as above, or you can call 
# separate functions to set up the parameter object
pars =  camb.set_params(H0=100*cosmos_LCDM['h'], ombh2=cosmos_LCDM['omega_b_0'], omch2=cosmos_LCDM['omega_d_0'], ns=cosmos_LCDM['ns'], mnu=0.0, omk=0,  
                       As=cosmos_LCDM['As'], halofit_version='mead2020', lmax=3000, tau = 0.06)

#Note non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=1000.0)

#Linear spectra
pars.NonLinear = model.NonLinear_none
results = camb.get_results(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1000, npoints = 820)
s8 = np.array(results.get_sigma8())

# #Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh_nonlin, z_nonlin, pk_nonlin = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1000, npoints = 200)

In [5]:
hmcode_params_LCDM_camb = HMcode_params.HMCode_param_dic(cosmos_LCDM, kh, pk[0,:])

PS_LCDM_matter_nonlin_axioncamb_PS = PS_nonlin_cold.func_non_lin_PS_matter(M_arr, power_spec_dic_LCDM['k'], power_spec_dic_LCDM['power_total'], 
                                                              cosmos_LCDM, hmcode_params_LCDM, cosmos_LCDM['Omega_m_0'],
                                                              alpha = True, 
                                                              eta_given = True, 
                                                              one_halo_damping = True,
                                                              two_halo_damping = True,
                                                              full_2h = False)

PS_LCDM_matter_nonlin_camb_PS = PS_nonlin_cold.func_non_lin_PS_matter(M_arr, kh, pk[0,:], 
                                                              cosmos_LCDM, hmcode_params_LCDM, cosmos_LCDM['Omega_m_0'],
                                                              alpha = True, 
                                                              eta_given = True, 
                                                              one_halo_damping = True,
                                                              two_halo_damping = True,
                                                              full_2h = False)
from scipy.interpolate import interp1d
axioncamb_interp_tot = interp1d(np.log(power_spec_dic_LCDM['k']), np.log(power_spec_dic_LCDM['power_total']))
axioncamb_interp_tot_nonlin_axioncamb_PS = interp1d(np.log(power_spec_dic_LCDM['k']), np.log(PS_LCDM_matter_nonlin_axioncamb_PS[0]))
axioncamb_interp_tot_nonlin_camb_PS = interp1d(np.log(kh), np.log(PS_LCDM_matter_nonlin_camb_PS[0]))


axioncamb_interp_1h_nonlin_axioncamb_PS = interp1d(np.log(power_spec_dic_LCDM['k']), np.log(PS_LCDM_matter_nonlin_axioncamb_PS[1]))
axioncamb_interp_2h_nonlin_axioncamb_PS = interp1d(np.log(power_spec_dic_LCDM['k']), np.log(PS_LCDM_matter_nonlin_axioncamb_PS[2]))

(0.95, 1.03)

In [6]:
plt.axhline(y = 1, ls = '--', color='k')
plt.semilogx(kh_nonlin,  np.exp(axioncamb_interp_tot_nonlin_axioncamb_PS(np.log(kh_nonlin))) /pk_nonlin[0,:], label='use axioncamb linear PS')
plt.semilogx(kh_nonlin,  np.exp(axioncamb_interp_tot_nonlin_camb_PS(np.log(kh_nonlin))) /pk_nonlin[0,:], label='from camb linear PS')
plt.legend()
plt.xlabel(r'$k\ [h/\mathrm{Mpc}]$')
plt.ylabel('axionHMcode/Hmcode from camb')
plt.ylim(0.95, 1.03)
# plt.savefig('comparison_to_HMcode_camb.pdf', dpi = 600)